In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, load_model

IMG_WIDTH = 300
IMG_HEIGHT = 300
MODEL_NAME = 'cnn-combined_updated.hd5'
model = load_model(MODEL_NAME)
img = Image.open('./coins_dataset/20c-new-heads/20c-image117.jpg')
img=img.rotate(4)
area = (400, 250, 950, 800)
img = img.crop(area)

pil_image = img.convert('RGB') 
open_cv_image = np.array(pil_image) 
# Convert RGB to BGR 
resized_image = open_cv_image[:, :, ::-1].copy() 
resized_image = cv2.resize(resized_image, (IMG_WIDTH, IMG_HEIGHT))
print(resized_image.shape)
resized_image = np.expand_dims(resized_image, axis=0)
print(resized_image.shape)

mapping_label = {'10c':0,
  '20c':1,
  '5c':2}

# model.predict(resized_image)
# print(resized_image)
y_prob = model.predict(resized_image) 
y_classes = y_prob.argmax(axis=-1)
y_classes

(300, 300, 3)
(1, 300, 300, 3)
1/1 [==============================] - 0s 350ms/step


array([1], dtype=int64)

In [2]:
data_dir = './test_dataset'
def load_data(dir):
    # initialise list of images
    images = []
    # initialise list of labels
    labels = []
    for root, dirs, files in os.walk(dir):
        
        for name in files:
            path = os.path.join(root, name)
            # use OpenCV to read each image as a numpy .ndarray (multidimensional array)
            img = Image.open(path)
            img=img.rotate(4)
            area = (400, 250, 950, 800)
            img = img.crop(area)
            
            pil_image = img.convert('RGB') 
            open_cv_image = np.array(pil_image) 
            # Convert RGB to BGR 
            resized_image = open_cv_image[:, :, ::-1].copy() 
            resized_image = cv2.resize(resized_image, (IMG_WIDTH, IMG_HEIGHT))
            resized_image = np.expand_dims(resized_image, axis=0)
            # add resized image to images list
            images.append(resized_image)
            labels.append(name)
            
    return images, labels
images, labels = load_data(data_dir)
print(labels)

['test-image1.jpg', 'test-image10.jpg', 'test-image2.jpg', 'test-image3.jpg', 'test-image4.jpg', 'test-image5.jpg', 'test-image6.jpg', 'test-image7.jpg', 'test-image8.jpg', 'test-image9.jpg']


In [3]:
for i in range(len(images)):
    
    y_prob = model.predict(images[i]) 
    y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
    print('Probability:',y_prob_dec)
    y_classes = y_prob.argmax(axis=-1)
    print('Predicted class:', y_classes, 'Filename:',labels[i], '\n')
    
mapping_label = {'10c':0,
  '20c':1,
  '5c':2}

1/1 [==============================] - 0s 33ms/step
Probability: ['0.00000012', '0.99999988', '0.00000001']
Predicted class: [1] Filename: test-image1.jpg 

1/1 [==============================] - 0s 32ms/step
Probability: ['0.00000001', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image10.jpg 

1/1 [==============================] - 0s 33ms/step
Probability: ['0.00000378', '0.99999607', '0.00000011']
Predicted class: [1] Filename: test-image2.jpg 

1/1 [==============================] - 0s 33ms/step
Probability: ['0.00008307', '0.99981219', '0.00010482']
Predicted class: [1] Filename: test-image3.jpg 

1/1 [==============================] - 0s 33ms/step
Probability: ['0.02310263', '0.97046340', '0.00643402']
Predicted class: [1] Filename: test-image4.jpg 

1/1 [==============================] - 0s 33ms/step
Probability: ['0.89719075', '0.07276656', '0.03004265']
Predicted class: [0] Filename: test-image5.jpg 

1/1 [==============================] - 0s 35ms/step
Proba

In [9]:
data_dir = './test_dataset'
mapping_label = {'10c':0,
  '20c':1,
  '5c':2}
def load_data(dir):
    # initialise list of images
    images = []
    # initialise list of labels
    labels = []
    for root, dirs, files in os.walk(dir):
        
        for name in files:
            path = os.path.join(root, name)
            # use OpenCV to read each image as a numpy .ndarray (multidimensional array)
            img = Image.open(path)
            img=img.rotate(4)
            area = (400, 250, 950, 800)
            img = img.crop(area)
            
            pil_image = img.convert('RGB') 
            open_cv_image = np.array(pil_image) 
            # Convert RGB to BGR 
            resized_image = open_cv_image[:, :, ::-1].copy() 
            # Convert to grayscale.
            gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

            # Blur using 3 * 3 kernel.
            gray_blurred = cv2.blur(gray, (3, 3))

            # Apply Hough transform on the blurred image.
            detected_circles = cv2.HoughCircles(gray_blurred, 
                               cv2.HOUGH_GRADIENT, 1, 50, param1 = 100,
                           param2 = 100, minRadius = 100, maxRadius = 300)

            # Draw circles that are detected.
            if detected_circles is not None:

                # Convert the circle parameters a, b and r to integers.
                detected_circles = np.uint16(np.around(detected_circles))

                for pt in detected_circles[0, :]:
                    a, b, r = pt[0], pt[1], pt[2]
                    print(a,b,r)
                    break
            
            # add resized image to images list
            images.append(r)
            labels.append(name)
    return images,labels
images, labels = load_data(data_dir)

226 324 190
252 296 189
252 298 186
284 236 196
282 312 200
228 268 169
288 186 162
202 334 165
280 374 152


In [10]:
import joblib
knn = joblib.load('knn_hough.pkl')
print(images[0].shape)
for i in range(len(images)):
#     nsamples, nx, ny, nrgb = images[i].shape
    x_test = images[i].reshape(-1,1)
    y_prob = knn.predict(x_test) 
    y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
    print('Probability:',y_prob_dec)
    y_classes = y_prob.argmax(axis=-1)
    print('Predicted class:', y_classes, 'Filename:',labels[i], '\n')

()
Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image1.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image10.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image2.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image3.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image4.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image5.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image6.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image7.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image8.jpg 

Probability: ['0.00000000', '0.00000000', '1.000000

In [5]:
import joblib
knn = joblib.load('knn.pkl')
print(images[0].shape)
for i in range(len(images)):
    nsamples, nx, ny, nrgb = images[i].shape
    x_test = images[i].reshape((nsamples,nx*ny*nrgb))
    y_prob = knn.predict(x_test) 
    y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
    print('Probability:',y_prob_dec)
    y_classes = y_prob.argmax(axis=-1)
    print('Predicted class:', y_classes, 'Filename:',labels[i], '\n')

(1, 300, 300, 3)
Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image1.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image10.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image2.jpg 

Probability: ['0.00000000', '0.00000000', '1.00000000']
Predicted class: [2] Filename: test-image3.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image4.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image5.jpg 

Probability: ['0.00000000', '0.00000000', '1.00000000']
Predicted class: [2] Filename: test-image6.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image7.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image8.jpg 

Probability: ['0.00000000', '0.000000

In [5]:
import joblib
rf = joblib.load('rf.pkl')
print(images[0].shape)
for i in range(len(images)):
    nsamples, nx, ny, nrgb = images[i].shape
    x_test = images[i].reshape((nsamples,nx*ny*nrgb))
    y_prob = rf.predict(x_test) 
    y_prob_dec = [format(float(prob), '.8f') for prob in y_prob[0]]
    print('Probability:',y_prob_dec)
    y_classes = y_prob.argmax(axis=-1)
    print('Predicted class:', y_classes, 'Filename:',labels[i], '\n')

(1, 300, 300, 3)
Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image1.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image10.jpg 

Probability: ['0.00000000', '1.00000000', '0.00000000']
Predicted class: [1] Filename: test-image2.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image3.jpg 

Probability: ['0.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image4.jpg 

Probability: ['0.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image5.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image6.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image7.jpg 

Probability: ['1.00000000', '0.00000000', '0.00000000']
Predicted class: [0] Filename: test-image8.jpg 

Probability: ['0.00000000', '0.000000